### 1. Import Library

In [1]:
from googleapiclient.discovery import build
import pandas as pd
from IPython.display import JSON
import numpy as np

# Connected Database
from sqlalchemy import create_engine
from getpass import getpass  # Não exibir a senha

# Vizualise data 
import seaborn as sns 
import matplotlib.pyplot as plt
%matplotlib inline

# view settings
sns.set_style('darkgrid')
plt.rcParams['font.size'] = 12
plt.rcParams['figure.figsize'] = (8, 5)

%load_ext sql   

### 2. Sttings API keys and Start Youtube

In [2]:
api_key = getpass()
api_service_name = "youtube"
api_version = "v3"

# Defined Charnnels
charnnel_ids = ['UC8mDF5mWNGE-Kpfcvnn0bUg',  #  Me poupe!
                'UCnBNoed6NaPlpL8blj1yy8Q',  # Clube do valor
                'UC_mSfchV-fgpPy-vuwML8_A',  # Gustavo Cerbasi
               ]

# Get credentials and create an API client
youtube = build(
    api_service_name, api_version, developerKey=api_key)

 ········


### 3. Get charnnel Statistic

In [3]:
def charnnel_stats(youtube, charnnel_ids):
    '''This function return the statistics of charnnels'''
    all_data = []
    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=",".join(charnnel_ids)
    )
    response = request.execute()

    for item in response['items']:
        data = {'Charnnel_name': item['snippet']['title'],
                'Subscribesrs': item['statistics']['subscriberCount'],
                'Viwes': item['statistics']['viewCount'],
                'Total_videos': item['statistics']['videoCount'],
                'Charnnel_ID': item['id'],
                'Playlist_ID': item['contentDetails']['relatedPlaylists']['uploads']
                 }
        all_data.append(data)
        
    return pd.DataFrame(all_data)

In [4]:
charnnel_df = charnnel_stats(youtube, charnnel_ids)

charnnel_df

,Charnnel_name,Subscribesrs,Viwes,Total_videos,Charnnel_ID,Playlist_ID
0,Clube do Valor,1040000,54831965,1594,UCnBNoed6NaPlpL8blj1yy8Q,UUnBNoed6NaPlpL8blj1yy8Q
1,Me Poupe!,7570000,547282805,1467,UC8mDF5mWNGE-Kpfcvnn0bUg,UU8mDF5mWNGE-Kpfcvnn0bUg
2,Gustavo Cerbasi,1020000,38634112,857,UC_mSfchV-fgpPy-vuwML8_A,UU_mSfchV-fgpPy-vuwML8_A


### 4. Get videos IDs

In [5]:
def get_videos_ids(youtube, playlist_id):
    '''This function returns all IDs of videos in the playlist.'''
    def fetch_page(playlist_id, page_token=None):
        request = youtube.playlistItems().list(
            part="snippet,contentDetails",
            playlistId=playlist_id,
            maxResults=50,
            pageToken=page_token
        )
        return request.execute()
    
    all_videos_id = []
    response = fetch_page(playlist_id)
    
    while response:
        for item in response['items']:
            video_id = {
                "Channel_ID": item['snippet']['channelId'],
                "Video_ID": item['contentDetails']['videoId']
            }
            all_videos_id.append(video_id)
        
        next_page_token = response.get('nextPageToken')
        if next_page_token:
            response = fetch_page(playlist_id, next_page_token)
        else:
            break

    return all_videos_id

In [6]:
playlist_ids = list(charnnel_df['Playlist_ID'])

In [7]:
all_videos_ids = []

for playlist_id in playlist_ids:
    videos_id = get_videos_ids(youtube, playlist_id)
    all_videos_ids.extend(videos_id)  

videos_df = pd.DataFrame(all_videos_ids)

In [8]:
videos_df.head()

,Channel_ID,Video_ID
0,UCnBNoed6NaPlpL8blj1yy8Q,4ZpZ1Gf6oFE
1,UCnBNoed6NaPlpL8blj1yy8Q,-cPblHGriXk
2,UCnBNoed6NaPlpL8blj1yy8Q,HDTTu2w9gF8
3,UCnBNoed6NaPlpL8blj1yy8Q,XCNPfH1Vq5Q
4,UCnBNoed6NaPlpL8blj1yy8Q,EXgttl5MAzs


### 5. Get videos Statistics

In [9]:
videos_ids = list(videos_df['Video_ID'][:5])

len(videos_ids)

5

In [10]:
def get_video_detail(youtube, video_ids):
    
    all_video_info = []

    for i in range(0, len(video_ids), 50):
        request = youtube.videos().list(
            part='snippet,contentDetails,statistics',
            id= ','.join(videos_ids[i:i+50])
        )
        response = request.execute()
    
        for video in response['items']:
            stats_to_keep = {'snippet': ['channelId', 'channelTitle','title', 'publishedAt'],
                             'statistics': ['viewCount', 'likeCount', 'favoriteCount','commentCount'],
                             'contentDetails': ['duration', 'caption']   
                            }
            video_info = {}
            video_info['video_id'] = video['id']
            
            for k in stats_to_keep.keys():
                for v in stats_to_keep[k]:
                    try:
                        video_info[v] = video[k][v]
                    except:
                        video_info[v] = None
    
            all_video_info.append(video_info)

    return pd.DataFrame(all_video_info)

In [11]:
video_info_df = get_video_detail(youtube, videos_ids)

In [12]:
video_info_df.head()

,video_id,channelId,channelTitle,title,publishedAt,viewCount,likeCount,favoriteCount,commentCount,duration,caption
0,4ZpZ1Gf6oFE,UCnBNoed6NaPlpL8blj1yy8Q,Clube do Valor,ELE PERDEU TUDO INVESTINDO (NÃO COMETA ESSE ERRO),2024-06-10T15:00:03Z,4486,570,0,20,PT12M22S,false
1,-cPblHGriXk,UCnBNoed6NaPlpL8blj1yy8Q,Clube do Valor,5 PASSOS PARA ATINGIR A LIBERDADE FINANCEIRA M...,2024-06-09T15:00:00Z,6040,462,0,15,PT17M24S,false
2,HDTTu2w9gF8,UCnBNoed6NaPlpL8blj1yy8Q,Clube do Valor,"AÇÕES ORDINÁRIAS, PREFERENCIAIS e UNITS: Qual ...",2024-06-08T15:00:16Z,5052,432,0,13,PT10M5S,false
3,XCNPfH1Vq5Q,UCnBNoed6NaPlpL8blj1yy8Q,Clube do Valor,POR QUE VOCÊ NÃO DEVE INVESTIR NO EXTERIOR?,2024-06-07T15:00:36Z,23943,1986,0,171,PT10M52S,false
4,EXgttl5MAzs,UCnBNoed6NaPlpL8blj1yy8Q,Clube do Valor,COMPROVADO: QUANTO INVESTIR PARA VIVER DE REND...,2024-06-06T15:00:18Z,4943,455,0,13,PT10M24S,false


### 6. Exploratory Data Analisys